In [23]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Dense, Conv2D, Conv2DTranspose

## At the end of the 3D Gaze Estimator

In [30]:
def planar_angular_loss(dx, dy, gx, gy):
    """
        d=(dx, dy) : ground truth planar gaze direction
        g=(gx, gy) : 2D projection of the estimated 3D gaze direction
    
    """
    
    res = 1 - (dx*gx+dy*gy)/(((dx**2 + dy**2)**(1/2))*(gx**2 + gy**2)**(1/2))    
    
    return res 

## Dual Attention Mode

In [1]:
def Dual_attention_mode(Mf, Md):
    """
        Mf : FOV   Attention Map
        Md : Depth Attention Map
        
        return -> Mdual : Dual Attention Map 
    """
    return Mf*Md

## Binary Classification Head

In [20]:
BCF_Model = Sequential()

nof_batch   = 1
nof_filters = 3
input_shape = (nof_batch, 224, 224, 3) #Channel last and concatenation of the Dual Attention Map and the scene image


BCF_Model.add(Conv2D(filters=nof_filters, kernel_size=(3,3), data_format='channels_last', activation='relu', padding="same", input_shape=input_shape[1:]))
BCF_Model.add(Conv2D(filters=nof_filters, kernel_size=(3,3), data_format='channels_last', activation='relu', padding="same"))
BCF_Model.add(Dense(2))
BCF_Model.summary()

# Compile model
BCF_Model.compile(loss='binary_crossentropy', optimizer='adam')

## Must have for output size 1 (two classes possible)



Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 224, 224, 3)       84        
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 224, 224, 3)       84        
_________________________________________________________________
dense_8 (Dense)              (None, 224, 224, 2)       8         
Total params: 176
Trainable params: 176
Non-trainable params: 0
_________________________________________________________________


## Heatmap Regression Head 

In [29]:
HRH_Model = Sequential()

nof_batch   = 1
nof_filters = 3
input_shape = (nof_batch, 224, 224, 3) #Channel last and concatenation of the Dual Attention Map and the scene image


HRH_Model.add(Conv2D(filters=nof_filters, kernel_size=(3,3), data_format='channels_last', activation='relu', padding="same", input_shape=input_shape[1:]))
HRH_Model.add(Conv2D(filters=nof_filters, kernel_size=(3,3), data_format='channels_last', activation='relu', padding="same"))
HRH_Model.add(Conv2DTranspose(filters=nof_filters, kernel_size=(3,3), padding='same', data_format='channels_last', activation='relu'))
HRH_Model.add(Conv2DTranspose(filters=nof_filters, kernel_size=(3,3), padding='same', data_format='channels_last', activation='relu'))
HRH_Model.add(Conv2DTranspose(filters=nof_filters, kernel_size=(3,3), padding='same', data_format='channels_last', activation='relu'))
HRH_Model.summary()

# Compile model
HRH_Model.compile(loss='mean_squared_error', optimizer='adam')


## Must have for output size (64,64)



Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 224, 224, 3)       84        
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 224, 224, 3)       84        
_________________________________________________________________
conv2d_transpose_15 (Conv2DT (None, 224, 224, 3)       84        
_________________________________________________________________
conv2d_transpose_16 (Conv2DT (None, 224, 224, 3)       84        
_________________________________________________________________
conv2d_transpose_17 (Conv2DT (None, 224, 224, 3)       84        
Total params: 420
Trainable params: 420
Non-trainable params: 0
_________________________________________________________________
